In [1]:
from datasets import load_dataset, Dataset
import pandas as pd
from tqdm import  tqdm

<h1> OASST <h1>

In [ ]:
oass_train = load_dataset("OpenAssistant/oasst2", split="train").to_pandas()
oass_valid = load_dataset("OpenAssistant/oasst2", split="validation").to_pandas()
oass_full = pd.concat([oass_train, oass_valid,])
oass_full.reset_index(drop=True, inplace=True)

In [ ]:
oass_full["review_result"].value_counts()

In [ ]:
oass_train = load_dataset("OpenAssistant/oasst2", split="train").to_pandas()
oass_valid = load_dataset("OpenAssistant/oasst2", split="validation").to_pandas()
oass_full = pd.concat([oass_train, oass_valid,])
oass_full.reset_index(drop=True, inplace=True)

needed_langs = ["en", "ar", "de", "es", "vi", "zh"]
rows = []
for lang in tqdm(needed_langs):
    print(f"Processing lang: {lang}")
    filtered_df = oass_full[(oass_full["lang"] == lang) & (oass_full["role"] == "assistant")]
    for i, answer in filtered_df.iterrows():
        query = oass_full[oass_full["message_id"] == answer["parent_id"]]["text"].iloc[0]
        rows.append([answer["lang"], answer["message_id"], answer["parent_id"], answer["user_id"], answer["created_date"], query, answer["text"], answer["review_count"]])
        
filtered_dataset = pd.DataFrame(rows, columns=["lang", "message_id", "parent_id", "user_id", "created_date", "query", "answer", "review_count"])
filtered_dataset.drop_duplicates(subset="answer", inplace=True)
filtered_dataset.reset_index(drop=True, inplace=True)

In [ ]:
hf_dataset = Dataset.from_pandas(filtered_dataset)
hf_dataset.push_to_hub("dkoterwa/oasst2_filtered_retrieval")

<h1> MKQA <h1>

In [ ]:
mkqa = load_dataset("mkqa", split="train").to_pandas()

In [ ]:
needed_langs = ["en", "ar", "de", "es", "vi", "zh_cn"]
rows = []
for i, row in tqdm(mkqa.iterrows(), total=mkqa.shape[0]):
    for lang in needed_langs:
        rows.append([lang, row["example_id"], row["queries"][lang], row["answers"][lang][0]["text"]])
        
filtered_dataset = pd.DataFrame(rows, columns=["lang", "example_id", "query", "answer"])
filtered_dataset.dropna(inplace=True)
filtered_dataset.reset_index(drop=True, inplace=True)

In [ ]:
hf_dataset = Dataset.from_pandas(filtered_dataset)
hf_dataset.push_to_hub("dkoterwa/mkqa_filtered")

<h1> MLQA <h1>

In [ ]:
def download_mlqa(subset_name):
    dataset_valid = load_dataset("mlqa", subset_name, split="validation").to_pandas()
    dataset_test  = load_dataset("mlqa", subset_name, split="test").to_pandas()
    full_dataset = pd.concat([dataset_valid, dataset_test])
    full_dataset.reset_index(drop=True, inplace=True)
    return full_dataset

needed_langs = ["mlqa.en.en", "mlqa.de.de", "mlqa.ar.ar", "mlqa.es.es", "mlqa.vi.vi", "mlqa.zh.zh"]
datasets = []
for lang in tqdm(needed_langs):
    dataset = download_mlqa(lang)
    dataset["lang"] = lang.split(".")[2]
    datasets.append(dataset)
    
full_mlqa = pd.concat(datasets)
full_mlqa.reset_index(drop=True, inplace=True)
full_mlqa["answer"] = [answer_dict["text"][0] for answer_dict in full_mlqa["answers"]]
full_mlqa.drop("answers", axis=1, inplace=True)

In [ ]:
hf_dataset = Dataset.from_pandas(full_mlqa)
hf_dataset.push_to_hub("dkoterwa/mlqa_filtered")

<h1> Database datasets <h1>

In [2]:
datasets = {"FreedomIntelligence/alpaca-gpt4-deutsch": "de",
            "FreedomIntelligence/alpaca-gpt4-spanish": "es",
            "FreedomIntelligence/alpaca-gpt4-chinese": "zh",
            "FreedomIntelligence/alpaca-gpt4-arabic": "ar"}

In [3]:
for dataset_name, language in datasets.items():
    dataset = load_dataset(dataset_name, split="train").to_pandas()
    dataset["answer"] = [conversation[1]["value"] for conversation in dataset["conversations"]]
    dataset["instruction"] = [conversation[0]["value"] for conversation in dataset["conversations"]]
    dataset.drop("conversations", axis=1, inplace=True)
    hf_dataset = Dataset.from_pandas(dataset)
    hf_dataset.push_to_hub(f"dkoterwa/alpaca_gpt_4_{language}")

Found cached dataset json (/teamspace/studios/this_studio/.cache/huggingface/datasets/FreedomIntelligence___json/FreedomIntelligence--alpaca-gpt4-deutsch-105d0824df499611/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


OSError: You need to provide a `token` or be logged in to Hugging Face with `huggingface-cli login`.

In [ ]:
dataset